In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))

def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(
        list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"]
    )
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

### List all XML Files in a folder

In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))


def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(
        list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"]
    )
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

### List all XML Files in a folder

In [ ]:
import glob
import os
import pandas as pd
from tqdm import tqdm

path = "/media/data1/muse_ge/ecg_retrospective"
all_files = glob.glob(os.path.join(path, "*.xml"))

# Convert all_files to a DataFrame with a progress bar
df = pd.DataFrame(tqdm(all_files, desc='Creating DataFrame'), columns=["path"])
display(df)

### Convert the 2 million ECGs into a PARQUET file 

In [ ]:
df_2 = pd.read_csv('/media/data1/anolin/added_label_box_2M.csv')

In [ ]:
## Drop df_2['RestingECG_PatientDemographics_PatientID'] that are not integers
df_2 = df_2[df_2['RestingECG_PatientDemographics_PatientID'].astype(str).str.isnumeric()]
### This line converts the 'RestingECG_PatientDemographics_PatientID' column to string type, then uses the str.isnumeric() function to filter out any rows where 'RestingECG_PatientDemographics_PatientID' is not numeric.

In [ ]:
df_2_grouped = df_2.groupby(['RestingECG_PatientDemographics_PatientID','RestingECG_TestDemographics_AcquisitionDate','RestingECG_TestDemographics_AcquisitionTime']).first().reset_index()

In [ ]:
import os 
# Assuming df is your DataFrame and 'path' is the column containing file paths
df_2_grouped['path_components'] = df_2_grouped['xml_path'].apply(lambda x: os.path.dirname(x))

# if df_2_grouped.path_components is nil add "data1/anolin/out_ECG_latest/xml"
df_2_grouped.loc[df_2_grouped.path_components == "", 'path_components'] = "/media/data1/muse_ge/ecg_retrospective"

## Get npy_path filename for df_2
df_2_grouped['xml_filename'] = df_2_grouped['xml_path'].apply(lambda x: os.path.basename(x))
df_2_grouped['xml_path'] = df_2_grouped['path_components'] + "/" + df_2_grouped['xml_filename']

## Create new dataframe called df3 if df_2_grouped.path_components is equal to /media/data1/anolin/out_ECG_latest/xml
df_2_grouped['path_components'] = df_2_grouped['xml_path'].apply(lambda x: os.path.dirname(x))
display(df_2_grouped['path_components'].value_counts())
################
### NPY ########
###############

## Get npy_path filename for df_2
df_2_grouped['npy_filename'] = df_2_grouped['npy_path'].apply(lambda x: os.path.basename(x))

## Create new dataframe called df3 if df_2_grouped.path_components is equal to /media/data1/anolin/out_ECG_latest/xml
df_2_grouped['new_npy_path'] = '/media/data1/ravram/DeepECG/ekg_waveforms_output/ecg_npy/' + df_2_grouped['npy_filename']
df_2_grouped['npy_path'] = df_2_grouped['new_npy_path']
## drop new_npy_path
df_2_grouped = df_2_grouped.drop(['new_npy_path'], axis=1)

## Create new dataframe called df3 if df_2_grouped.path_components is equal to /media/data1/anolin/out_ECG_latest/xml
df_2_grouped['npy_path_components'] = df_2_grouped['npy_path'].apply(lambda x: os.path.dirname(x))
display(df_2_grouped['npy_path_components'].value_counts())

In [ ]:
## drop npy_path_components and path_components
df_2_grouped = df_2_grouped.drop(['npy_path_components','path_components'], axis=1)

In [ ]:
df_2_grouped['RestingECG_PatientDemographics_PatientID_int'] = df_2_grouped['RestingECG_PatientDemographics_PatientID'].astype(int)

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# Assuming df_2_grouped is your DataFrame and 'new_xml_path' is the column containing file paths

for file_path in tqdm(df_2_grouped['xml_path']):
    if os.path.exists(file_path):
        continue
    else:
        print(f"The file {file_path} does not exist.")
        break
df_2_grouped['file_exists'] = df_2_grouped['xml_path'].apply(os.path.exists)

In [ ]:
display(df_2_grouped.RestingECG_PatientDemographics_PatientID.count())

In [ ]:
## Clean the data


df_2_grouped['Normalized_Diagnosis_LABELBOX'] = df_2_grouped['Normalized_Diagnosis_LABELBOX'].astype(str)
#df_2_grouped = df_2_grouped.drop(['RestingECG_PatientDemographics_PatientFirstName'], axis=1)
#df_2_grouped = df_2_grouped.drop(['RestingECG_PatientDemographics_PatientLastName'], axis=1)

df_2_grouped['RestingECG_PatientDemographics_PatientID'] = df_2_grouped['RestingECG_PatientDemographics_PatientID'].astype(str)
df_2_grouped['RestingECG_TestDemographics_AnalysisSoftwareVersion'] = df_2_grouped['RestingECG_TestDemographics_AnalysisSoftwareVersion'].astype(str)

In [ ]:
# Convert the column to string type
df_2_grouped['RestingECG_TestDemographics_AnalysisSoftwareVersion'] = df_2_grouped['RestingECG_TestDemographics_AnalysisSoftwareVersion'].astype(str)

# Replace all non-digit characters with an empty string
df_2_grouped['RestingECG_TestDemographics_AnalysisSoftwareVersion'] = df_2_grouped['RestingECG_TestDemographics_AnalysisSoftwareVersion'].str.replace(r'\D', '', regex=True)
# Convert empty strings to NaN
df_2_grouped.loc[df_2_grouped['RestingECG_TestDemographics_AnalysisSoftwareVersion'] == '', 'RestingECG_TestDemographics_AnalysisSoftwareVersion'] = np.nan

df_2_grouped['RestingECG_TestDemographics_AnalysisSoftwareVersion'] = df_2_grouped['RestingECG_TestDemographics_AnalysisSoftwareVersion'].astype(float).astype('Int64')
display(df_2_grouped['RestingECG_TestDemographics_AnalysisSoftwareVersion'].value_counts())

In [ ]:
df_2_grouped['RestingECG_Order_ExtraADTData1'] = df_2_grouped['RestingECG_Order_ExtraADTData1'].astype(str)
# Iterate over the columns in the DataFrame
for col in df_2_grouped.columns:
    # If the column name contains "RestingECG_ExtraQuestions"
    if "RestingECG_ExtraQuestions" in col:
        # Convert the column to string type
        df_2_grouped[col] = df_2_grouped[col].astype(str)
    elif "RestingECG_Order" in col:
        df_2_grouped[col] = df_2_grouped[col].astype(str)

In [ ]:
df_2_grouped.to_parquet(
    "/media/data1/ravram/DeepECG/ekg_waveforms_output/df_xml_2023_05_09_2004-to-june-2022_n_1572280_with_labelbox_no_duplicates.parquet"
)

### Plot dataset distribution

In [ ]:
df_2_grouped = pd.read_parquet('/media/data1/ravram/DeepECG/ekg_waveforms_output/df_xml_2023_05_09_2004-to-june-2022_n_1572280_with_labelbox_no_duplicates.parquet')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert the 'RestingECG_TestDemographics_AcquisitionDate' column to datetime
df_2_grouped['RestingECG_TestDemographics_AcquisitionDate'] = pd.to_datetime(df_2_grouped['RestingECG_TestDemographics_AcquisitionDate'])

# Extract the year and count the number of entries per year
entries_per_year = df_2_grouped['RestingECG_TestDemographics_AcquisitionDate'].dt.year.value_counts()

# Sort the years
entries_per_year = entries_per_year.sort_index()

# Plot the number of entries per year
entries_per_year.plot(kind='bar', figsize=(12,6))

plt.xlabel('Year')
plt.ylabel('Number of Entries')
plt.title('Number of Entries per Year')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert the 'RestingECG_TestDemographics_AcquisitionDate' column to datetime
df_2['RestingECG_TestDemographics_AcquisitionDate'] = pd.to_datetime(df_2['RestingECG_TestDemographics_AcquisitionDate'])

# Filter the dataframe for entries since 2018
df_2 = df_2[df_2['RestingECG_TestDemographics_AcquisitionDate'].dt.year >= 2018]

# Create new columns for year and month
df_2['Year'] = df_2['RestingECG_TestDemographics_AcquisitionDate'].dt.year
df_2['Month'] = df_2['RestingECG_TestDemographics_AcquisitionDate'].dt.month

# Group by year and month, and count the number of entries
entries_per_month = df_2.groupby(['Year', 'Month']).size()

# Plot the number of entries per month for each year
entries_per_month.plot(kind='bar', figsize=(12,6))

plt.xlabel('Year, Month')
plt.ylabel('Number of Entries')
plt.title('Number of Entries per Month (Since 2018)')
plt.show()

In [ ]:
# df_output['ecg_abnormal'] = np.where(df_output['Diagnosis'].str.contains('ECG anormal'), 1, np.where(df_output['Diagnosis'].str.contains('ECG normal'), 0, -1))
# Remove ECG anormal and ECG normal from diagnosis
# df_output['Diagnosis'] = df_output['Diagnosis'].str.replace('ECG anormal', '')
# df_output['Original_Diagnosis'] = df_output['Original_Diagnosis'].str.replace('ECG normal', '')
# df_output['Original_Diagnosis'] = df_output['Diagnosis'].str.replace('ECG anormal', '')
# df_output['Diagnosis'] = df_output['Original_Diagnosis'].str.replace('ECG normal', '')
# df_output.to_csv('data/20221002_ECG_mod_diagnosis.csv')

In [ ]:
df_m = (
    df_output.groupby(["patientid", "AcquisitionDate", "AcquisitionTime"])
    .first()
    .reset_index()
)
display(df_stats(df_output))
display(df_stats(df_m))
## The ECGs can be grouped by patient id, date and time to have a 1 unique row per ECG - this means the filename to save the ECG also needs to have the date and time in it.

In [ ]:
# output_dir = os.getcwd() + '/ekg_waveforms_output/'
# ekg_batch_run(df['path'][0:1])

###  Run CLI_xml2df by Alexis to generate NUMPY ARRAYS and SAVE dataset as PARQUET

In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))


import os

os.environ["R_HOME"] = "/root/miniconda3/envs/R/lib/R"

import rpy2.robjects as objects
from rpy2.robjects.packages import importr

base = importr("base")
r_pROC = importr("pROC")
base._libPaths()[0]

# Group by year and month, and count the number of entries
entries_per_month = df_2.groupby(['Year', 'Month']).size()

def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(
        list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"]
    )
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

In [ ]:
df = pd.read_csv("data/20230313_ECG_path.csv")

In [ ]:
import CLI_xml2df as xml2df

df_output = xml2df.tinyxml2df(
    df["path"], out_path="data/ekg_waveforms_output/"
).read2flatten()

In [ ]:
#df_output.to_csv('data/20230328_ECG_path_diagnosis_fixed_with_NPY.csv')

In [ ]:
df_output.to_parquet(
    "/media/data1/ravram/DeepECG/ekg_waveforms_output/df_xml_2023_03_30_n_1633856.parquet"
)

In [ ]:
npy_array = np.load(df_output["npy_path"][0])
# Transpose the array to shape=[12, 2500]
ecg_transposed = np.transpose(npy_array, (1, 0, 2))
ecg_transposed = ecg_transposed.reshape(12, 2500)

import ecg_plot

ecg_plot.plot(ecg_transposed, sample_rate=250, title="ECG 12")
ecg_plot.show()

In [ ]:
## Plot line of ecg_transposed[0] and ecg_transposed[1]
import matplotlib.pyplot as plt

plt.plot(ecg_transposed[0])
plt.plot(ecg_transposed[1])
plt.show()

### Generate ECGs for LABELBOX ANNOTATION - TOP 3600 DIAGNOSES

In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import re

import pandas as pd
from unidecode import unidecode


def normalize_string(s):
    s = unidecode(s)
    s = s.strip()
    s = re.sub(r"\W+", " ", s)
    return s


def contains_eligible_diagnosis(normalized_diag, eligible_diagnoses):
    # Initialize a variable to store the longest diagnosis and its length
    longest_diagnosis = None
    longest_length = 0

    # Iterate over eligible_diagnoses
    for diagnosis in eligible_diagnoses:
        # Check if diagnosis is in normalized_diag
        if diagnosis in normalized_diag:
            # Check if the length of the current diagnosis is greater than the current longest_length
            if len(diagnosis) > longest_length:
                # Update the longest_diagnosis and longest_length
                longest_diagnosis = diagnosis
                longest_length = len(diagnosis)
                
    if (longest_diagnosis != None):
        # Return the longest_diagnosis
        return longest_diagnosis
    else:
        return False


def filter_ecg_by_diagnosis(df_ecg, df_ecg_sampled):
    df_ecg["Normalized_Diagnosis_MUSE"] = df_ecg["diagnosis"].apply(normalize_string)
    df_ecg_sampled["Normalized_Diagnosis_LABELBOX"] = df_ecg_sampled["Original_Diagnosis"].apply(
        normalize_string
    )

    eligible_diagnoses = set(df_ecg_sampled["Normalized_Diagnosis_LABELBOX"])

    df_ecg["Normalized_Diagnosis_LABELBOX"] = df_ecg["Normalized_Diagnosis_MUSE"].apply(
        lambda x: contains_eligible_diagnosis(x, eligible_diagnoses)
    )

    return df_ecg

In [ ]:
df_ecg = pd.read_parquet(
    "/media/data1/ravram/DeepECG/ekg_waveforms_output/df_xml_2023_03_30_n_1633856.parquet"
)

In [ ]:
df_ecg = pd.read_parquet(
    "/media/data1/ravram/DeepECG/ekg_waveforms_output/df_xml_2023_03_30_n_1633856.parquet"
)

In [ ]:
# Replace the folder path in the npy_path column
#df_ecg['npy_path'] = df_ecg['npy_path'].replace(
#    r'data/ekg_waveforms_output/ecg_npy/', 
#    '/media/data1/ravram/DeepECG/ekg_waveforms_output/ecg_npy/', 
#    regex=True
#)

In [ ]:
import os
# Check if df_ecg['npy_path'] exists
display(df_ecg.npy_path)#
df_ecg[0:10000]['npy_path'].apply(lambda x: os.path.exists(x)).value_counts()

In [ ]:
df_ecg_sampled = pd.read_csv("data/20221002_ECG_mod_diagnosis_sampled_3600.csv")
df_ecg_sampled = df_ecg_sampled.groupby(['Diagnosis']).first()
display(df_ecg_sampled.describe())

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


def save_ecg_plots(df_output_sampled, output_dir, diagonsis_column='diagnosis', figsize=(16, 9), dpi=300, lead_order=None):
    if lead_order is None:
        lead_order = [
            "I",
            "II",
            "III",
            "aVR",
            "aVL",
            "aVF",
            "V1",
            "V2",
            "V3",
            "V4",
            "V5",
            "V6",
        ]

    path_to_output = os.path.join(os.getcwd(), output_dir)
    
    if not os.path.exists(path_to_output):
        os.mkdir(path_to_output)

    for index, row in tqdm(df_output_sampled.iterrows()):
        path = os.path.join(row["npy_path"])
        file = np.load(path)
        file = np.reshape(file, (1, 2500, 12))

        plt.rcParams["figure.figsize"] = figsize
        plt.ioff()
        fig, axs = plt.subplots(len(lead_order))
        
        for i in range(12):
            if i == 0:
                axs[i].set_title(row[diagonsis_column])
            axs[i].plot(file[0][:, i])
            axs[i].set(ylabel=str(lead_order[i]))

        AcquisitionDateTime = row["RestingECG_TestDemographics_AcquisitionDate"] + "_" + row["RestingECG_TestDemographics_AcquisitionTime"].replace(":", "-")
        filename = f"{row['RestingECG_PatientDemographics_PatientID']}_{AcquisitionDateTime}.png"
        
        file_output = os.path.join(path_to_output, filename)
        print(file_output)
        
        plt.savefig(file_output, dpi=dpi, bbox_inches="tight")
        plt.close(fig)df_ecg_sampled = pd.read_csv("data/20221002_ECG_mod_diagnosis_sampled_3600.csv")
df_ecg_sampled = df_ecg_sampled.groupby(['Diagnosis']).first()
display(df_ecg_sampled.describe())

In [ ]:
#save_ecg_plots(df_f[6511:6513], output_dir="sanity_check/")import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


def save_ecg_plots(df_output_sampled, output_dir, diagonsis_column='diagnosis', figsize=(16, 9), dpi=300, lead_order=None):
    if lead_order is None:
        lead_order = [
            "I",
            "II",
            "III",
            "aVR",
            "aVL",
            "aVF",
            "V1",
            "V2",
            "V3",
            "V4",
            "V5",
            "V6",
        ]

    path_to_output = os.path.join(os.getcwd(), output_dir)
    
    if not os.path.exists(path_to_output):
        os.mkdir(path_to_output)

    for index, row in tqdm(df_output_sampled.iterrows()):
        path = os.path.join(row["npy_path"])
        file = np.load(path)
        file = np.reshape(file, (1, 2500, 12))

        plt.rcParams["figure.figsize"] = figsize
        plt.ioff()
        fig, axs = plt.subplots(len(lead_order))
        
        for i in range(12):
            if i == 0:
                axs[i].set_title(row[diagonsis_column])
            axs[i].plot(file[0][:, i])
            axs[i].set(ylabel=str(lead_order[i]))

        AcquisitionDateTime = row["RestingECG_TestDemographics_AcquisitionDate"] + "_" + row["RestingECG_TestDemographics_AcquisitionTime"].replace(":", "-")
        filename = f"{row['RestingECG_PatientDemographics_PatientID']}_{AcquisitionDateTime}.png"
        
        file_output = os.path.join(path_to_output, filename)
        print(file_output)
        
        plt.savefig(file_output, dpi=dpi, bbox_inches="tight")
        plt.close(fig)

In [ ]:
#save_ecg_plots(df_f[6511:6513], output_dir="sanity_check/")

In [ ]:
pd.options.display.max_colwidth = 1000

In [ ]:
df_f = filter_ecg_by_diagnosis(df_ecg, df_ecg_sampled)

In [ ]:
## Apply this if the "Contains_Eligible_Diagnosis" is not FALSE; if it is false, return 0
df_f['percentage_labelbox_string_present_in_original_string'] = df_f.apply(lambda x: len(x['Normalized_Diagnosis_LABELBOX'])/len(x['diagnosis']) if x['Normalized_Diagnosis_LABELBOX'] != False else 0, axis=1)

In [ ]:
display(df_f.percentage_labelbox_string_present_in_original_string.describe())

In [ ]:
df_f['Normalized_Diagnosis_LABELBOX'] = df_f['Normalized_Diagnosis_LABELBOX'].astype(str)
df_f = df_f.drop(['RestingECG_PatientDemographics_PatientFirstName'], axis=1)
df_f = df_f.drop(['RestingECG_PatientDemographics_PatientLastName'], axis=1)

In [ ]:
df_f['Normalized_Diagnosis_LABELBOX'] = df_f['Normalized_Diagnosis_LABELBOX'].astype(str)
df_f = df_f.drop(['RestingECG_PatientDemographics_PatientFirstName'], axis=1)
df_f = df_f.drop(['RestingECG_PatientDemographics_PatientLastName'], axis=1)

In [ ]:
df_f.to_parquet(
    "/media/data1/ravram/DeepECG/ekg_waveforms_output/df_xml_2023_03_30_n_1633856_with_labelbox.parquet"
)

In [ ]:
### 1) Voir quelles données labelbox sont discordantes et me les envoyer pour annotation (et enlever ces ECG de la base de données)
### 2) Faire propagation des données labelbox concordantes sur df_ecg (db. parquet)
### 3) Dictionnaire pour df_ecg['Contains_Eligible_Diagnosis']==False
### 4) Standardiser le dataloader